In [1]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
import folium
import random
from src.functions import connectCollection
from src.functions import getCompaniesNear

In [2]:
# {$and:[{},{}]}
# {$and:[{$or:[{},{},{},{}]},{}]}

In [3]:
db, coll = connectCollection('companies','companies_cleaned')

In [4]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies')

In [5]:
coll

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies'), 'companies_cleaned')

In [6]:
companies = list(coll.find())

In [7]:
len(companies) # 9025

9025

In [8]:
categories = []
for i in range(len(companies)):
    categories.append(companies[i]['category_code'])
print(set(categories))

{'games_video', 'semiconductor', 'other', 'local', 'advertising', 'hospitality', 'automotive', 'mobile', 'network_hosting', 'ecommerce', 'photo_video', 'transportation', 'cleantech', 'nonprofit', 'consulting', 'enterprise', 'social', 'medical', 'manufacturing', 'messaging', 'finance', 'design', 'education', 'health', 'public_relations', 'news', 'analytics', 'real_estate', None, 'hardware', 'security', 'nanotech', 'biotech', 'travel', 'sports', 'web', 'music', 'legal', 'fashion', 'software', 'search'}


In [9]:
# Nobody in the company likes to have companies with more than 10 years in a radius of 2 KM.
# Developers like to be near successful tech startups that have raised at least 1 Million dollars.
# 30% of the company have at least 1 child.
# Executives like Starbucks A LOT. Ensure there's a starbucks not to far.
# Account managers need to travel a lot
# All people in the company have between 25 and 40 years, give them some place to go to party.
# The CEO is Vegan

In [10]:
active_companies = list(coll.find({'deadpooled_year': np.nan}))

In [11]:
len(active_companies) # 8406

8406

In [139]:
# Nobody in the company likes to have companies with more than 10 years in a radius of 2 KM.

# {$and:[{deadpooled_year: NaN},{founded_year:{$lte:2009}}]}

# old_companies = list(coll.find({'$and':[{'deadpooled_year': np.nan},{'founded_year':{'$lte':2009}}]}))

# 13 years
old_companies = list(coll.find({'$and':[{'deadpooled_year': np.nan},{'founded_year':{'$lte':2016}}]}))

# not near

In [202]:
years= 13
old_companies = list(coll.find({'$and':[{'deadpooled_year': np.nan},{'founded_year':{'$lte':2019-years}}]}))

In [203]:
len(old_companies) # 6840

4082

In [94]:
# Developers like to be near successful tech startups that have raised at least 1 Million dollars.

# {$and:[{$or:[{category_code:'hardware'},{category_code:'nanotech'},{category_code:'mobile'},
# {category_code:'games_video'},{category_code:'cleantech'},{category_code:'software'},{category_code:'analytics'},
# {category_code:'web'},{category_code:'biotech'}]},{deadpooled_year: NaN},{founded_year:{$gte:2010}},
# {total_money_raised_USD:{$gte:1000000}}]}

# http://www.startup-book.com/2016/05/22/when-was-the-word-start-up-first-used/

successful_tech_startups = list(coll.find({'$and':[{'$or':[
        {'category_code':'semiconductor'},{'category_code':'network_hosting'},{'category_code':'ecommerce'},
        {'category_code':'photo_video'},{'category_code':'consulting'},{'category_code':'design'},
        {'category_code':'automotive'},{'category_code':'hardware'},{'category_code':'nanotech'},
        {'category_code':'mobile'},{'category_code':'security'},{'category_code':'games_video'},
        {'category_code':'cleantech'},{'category_code':'software'},{'category_code':'analytics'},
        {'category_code':'web'},{'category_code':'biotech'}]},
    {'deadpooled_year': np.nan},{'founded_year':{'$gte':1976}},{'total_money_raised_USD':{'$gte':1000000}}]}))


In [95]:
print(len(successful_tech_startups))

1366


In [16]:
# Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.
design_companies = list(coll.find({'category_code':'design'}))

In [17]:
print(len(design_companies))

2


In [ ]:
# https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude/43211266#43211266
# import geopy.distance
# coords_1 = (52.2296756, 21.0122287)
# coords_2 = (52.406374, 16.9251681)
# print geopy.distance.vincenty(coords_1, coords_2).km

In [96]:
# Developers like to be near successful tech startups that have raised at least 1 Million dollars.
possible_offices_criterion_1 = []
for e in successful_tech_startups:
    near_companies = getCompaniesNear(e['office_1_location']['coordinates'][0],
                                      e['office_1_location']['coordinates'][1],2000)
    # [near_companies[i] for i in range(len(near_companies)) if near_companies[i]['name'] == e['name']]
    # [near_companies[i] for i in range(len(near_companies)) if near_companies[i]['name'] != e['name']]
    for f in near_companies:
        if f['_id'] != e['name']:
            possible_offices_criterion_1.append(f['_id'])

In [97]:
print(len(list(set(possible_offices_criterion_1)))) # 5103 oficinas tienen cerca una startup

5103


In [98]:
possible_offices_c1 = list(set(possible_offices_criterion_1))

In [141]:
# Nobody in the company likes to have companies with more than 10 years in a radius of 2 KM.

not_possible_offices_criterion_2 = []
for e in old_companies:
    near_companies = getCompaniesNear(e['office_1_location']['coordinates'][0],
                                      e['office_1_location']['coordinates'][1],2000)
    for f in near_companies:
        if f['_id'] != e['name']:
            not_possible_offices_criterion_2.append(f['_id'])

In [142]:
print(len(list(set(not_possible_offices_criterion_2)))) # 8476/7358 oficinas tienen cerca una compañía antigua: 10/13 años

7358


In [143]:
not_possible_offices_c2 = list(set(not_possible_offices_criterion_2))

In [181]:
possible_offices_c1_c2 = []
for e in possible_offices_c1:
    if e not in not_possible_offices_c2:
        possible_offices_c1_c2.append(e)        
possible_offices_c1_c2_to_string = [str(e) for e in possible_offices_c1_c2]

In [182]:
print(len(possible_offices_c1_c2))
print(len(possible_offices_c1_c2_to_string))

93
93


In [ ]:
df = pd.read_csv('./input/companies_df.csv')

cols = ['_id', 'name', 'founded_year', 'category_code','deadpooled_year', 'total_money_raised_USD', 'num_offices',
 'office_1_longitude','office_1_latitude', 'office_1_location', 'office_1_city','office_1_state_code',
 'office_1_country_code']
df = df[cols]

In [185]:
indexs = []
for i in range(len(df)):
    if df['_id'][i] in possible_offices_c1_c2_to_string:
        indexs.append(i)
print(len(indexs))
print(indexs)

93
[454, 478, 669, 736, 755, 952, 1376, 1377, 1519, 1624, 1696, 1766, 1920, 2084, 2143, 2161, 2613, 2670, 2695, 2697, 2823, 2878, 2880, 3028, 3133, 3232, 3258, 3590, 3673, 3677, 3753, 3794, 3974, 3991, 4146, 4186, 4367, 4384, 4487, 4544, 4632, 4858, 4885, 4900, 4956, 5017, 5158, 5231, 5241, 5374, 5593, 5837, 5874, 5949, 6195, 6338, 6413, 6671, 6940, 6968, 6999, 7024, 7042, 7255, 7386, 7395, 7468, 7610, 7668, 7812, 7877, 7904, 7920, 7928, 8000, 8060, 8081, 8110, 8190, 8218, 8263, 8285, 8308, 8408, 8445, 8590, 8591, 8620, 8675, 8745, 8762, 8792, 8892]


In [186]:
df_filtered = df.iloc[indexs]

In [187]:
df_filtered.head()

,_id,name,founded_year,category_code,deadpooled_year,total_money_raised_USD,num_offices,office_1_longitude,office_1_latitude,office_1_location,office_1_city,office_1_state_code,office_1_country_code
454,52cdef7c4bab8bd675297f9e,GENWI,2010.0,mobile,NaN,7100000.0,1,-111.903500,33.817100,"{'type': 'Point', 'coordinates': [-111.9035, 3...",Los Altos,CA,USA
478,52cdef7c4bab8bd675297fc4,ChessPark,2007.0,games_video,NaN,1000000.0,1,-83.913153,43.670840,"{'type': 'Point', 'coordinates': [-83.913153, ...",West Palm Beach,CA,USA
669,52cdef7c4bab8bd6752980be,Cyberhomes,NaN,web,NaN,0.0,1,-81.674745,30.318112,"{'type': 'Point', 'coordinates': [-81.674745, ...",Jacksonville,FL,USA
736,52cdef7c4bab8bd67529811d,RedLasso,NaN,games_video,NaN,8500000.0,1,-75.395990,40.099703,"{'type': 'Point', 'coordinates': [-75.39599, 4...",King of Prussia,PA,USA
755,52cdef7c4bab8bd675298139,divorce360,2007.0,web,NaN,2500000.0,1,-80.058401,26.820375,"{'type': 'Point', 'coordinates': [-80.058401, ...",North Palm Beach,FL,USA


In [188]:
possible_offices_c1_c2_coords = []
for e in companies:
    for f in possible_offices_c1_c2:
        if e['_id'] == f:
            possible_offices_c1_c2_coords.append(e['office_1_location'])

In [189]:
print(len(possible_offices_c1_c2_coords))

93


In [138]:
# for e in possible_offices_c1_c2_coords:
    # print(e['coordinates'][::-1])

In [201]:
# map_city = folium.Map(location=[40.7221,-73.9712], zoom_start=12)
# for company in manhattan_companies:
    # folium.Marker(company['office_1_location']['coordinates'][::-1],radius=2,
                  # icon=folium.Icon(icon='cloud',color='red')).add_to(map_city)
# map_city

tooltip = 'Click me!'
e = possible_offices_c1_c2_coords[random.choice(range(0,len(possible_offices_c1_c2_coords)))]
map_city = folium.Map(location = e['coordinates'][::-1], zoom_start=13)
folium.Circle(radius=2000,location=e['coordinates'][::-1],popup='Old companies free zone',color='#3186cc',
    fill=True,fill_color='#3186cc').add_to(map_city)
folium.Marker(e['coordinates'][::-1],radius=2,icon=folium.Icon(
    icon='briefcase',color='red'),popup='<b>Perfect location for your business</b>',
    tooltip=tooltip).add_to(map_city)
map_city
# map_city.save('index.html')